In [ ]:
# HOME_POSE = np.array([-0.3660424012449587, 0.6117162981347448, 0.9079274150612482, -0.005400995019868388, -0.0019643877311394183, -2.354619210713495,])

# # EMPIRACLLY FOUND BOUNDS LMAOOO
# # X_1 0.9424930299658596 0.6117557194238993 0.9078604285583273 -0.00548249567674523 -0.001965080545549638 -2.354599543904734
# # X_2 -0.9550680520089313 0.6117414180340556 0.9079711187002182 -0.0054476943775778695 -0.0019114211088729508 -2.354654218591263
# # Y_1 -0.3660752951621683 -0.9769698635830291 0.9078264973478847 -0.0053542629496214685 -0.001810702334644549 -2.3546477885984776
# # Y_2 -0.36601869093216893 1.0760272286463097 0.9079063215858276 -0.005390598302379138 -0.0020253279548708295 -2.3546219602941947
# # Z_1 -0.3660286022964023 0.6117127454372178 0.19288964682311052 -0.005344021094379553 -0.0020067450336018183 -2.3546172810944115
# # Z_2 -0.36607992182429566 0.611725276577193 1.3242422427347065 -0.005394167984549681 -0.00198225913263719 -2.35459296105138

# # ps = [
# #     np.array([0.9424930299658596, 0.6117557194238993, 0.9078604285583273,]),
# #     np.array([-0.9550680520089313, 0.6117414180340556, 0.9079711187002182,]),
# #     np.array([-0.3660752951621683, -0.9769698635830291, 0.9078264973478847,]),
# #     np.array([-0.36601869093216893, 1.0760272286463097, 0.9079063215858276,]),
# #     np.array([-0.3660286022964023, 0.6117127454372178, 0.19288964682311052,]),
# #     np.array([-0.36607992182429566, 0.611725276577193, 1.3242422427347065,]),
# # ]

import preamble
import numpy as np
from robot import Robot
from timer import Timer
from phrase_trial_data import PhraseTrialData
import random
import time
from virtual_dynamics import SimpleVirtualDynamics

NUMBER_OF_USERS = 12
NUMBER_OF_TRIALS_PER_USER = 42

USER_ID = 1

current_trial_number = 1

NUM_OF_STAGES = 5
stage = 0
def next_stage():
    global stage
    stage = (stage + 1) % NUM_OF_STAGES

RADIUS = 0.35

class ForceCurve:
    def __init__(self, direction, duration, peak_force, ramp_up_pct=0.5, hold_pct=0.0, ramp_down_pct=0.5):
        self.direction = direction
        self.ramp_up_time = ramp_up_pct * duration
        self.hold_time = hold_pct * duration
        self.ramp_down_time = ramp_down_pct * duration
        self.duration = duration
        self.peak_force = peak_force
        self.start_time = 0.0
        self.stop_time = 0.0
    
    def start(self):
        self.start_time = time.time()
    
    def stop(self):
        self.stop_time = time.time()
    
    def get_force(self):
        t = time.time() - self.start_time
        if t <= self.ramp_up_time:
            magnitude = self.peak_force * t / self.ramp_up_time
        elif t <= self.hold_time:
            magnitude = self.peak_force
        elif t <= self.ramp_down_time:
            magnitude = self.peak_force * (1.0 - (t - self.ramp_up_time - self.hold_time) / self.ramp_down_time)
        else:
            magnitude = 0.0
        
        is_done = t > self.duration
        if is_done:
            self.stop()

        return self.direction * magnitude, is_done
    
ANGLE = 5.0 * np.pi / 4.0
TRANSFORMATION = np.array([
    [np.cos(ANGLE), -np.sin(ANGLE), 0.0],
    [np.sin(ANGLE),  np.sin(ANGLE), 0.0],
    [0.0, 0.0, 1.0]
])
    
CARTESIAN_DIRECTION_VECTORS = [
    np.array([-1,0,0]),
    np.array([1,0,0]),
    np.array([0,-1,0]),
    np.array([0,1,0]),
    np.array([0,0,-1]),
    np.array([0,0,1]),
]
RANDOM_PEAK_BOUNDS = (5.0, 50.0)
BASELINE_PEAK = sum(RANDOM_PEAK_BOUNDS)/len(RANDOM_PEAK_BOUNDS)
RANDOM_DURATION_BOUNDS = (0.5, 1.5)
BASELINE_DURATION = sum(RANDOM_DURATION_BOUNDS)/len(RANDOM_DURATION_BOUNDS)

HOME_POSE = np.array([-0.3660424012449587, 0.6117162981347448, 0.9079274150612482, -0.005400995019868388, -0.0019643877311394183, -2.354619210713495,])

with Robot(
    '169.254.9.43',
    translational_force_deadband=6.0,
    rotational_force_deadband=0.5,
    init_pose=HOME_POSE
) as r:
    AXES = Robot.TRANSLATION

    timer = Timer()
    virtual_dynamics = SimpleVirtualDynamics(M=10.0, B=25.0, K=0.0)

    while current_trial_number <= NUMBER_OF_TRIALS_PER_USER:
        period_start = r.control.initPeriod()

        if stage == 0: # wait to begin trial
            cmd = input()
            if cmd == 'exit':
                break
            elif cmd == '':
                continue
            elif cmd == 'c':
                pass
            next_stage()
        elif stage == 1: # trial initialization
            phrase_trial_data = PhraseTrialData(USER_ID, current_trial_number, '', '', '', '')

            if current_trial_number in range(1,len(CARTESIAN_DIRECTION_VECTORS)+1):
                force_curve = ForceCurve(CARTESIAN_DIRECTION_VECTORS[current_trial_number-1] @ TRANSFORMATION.T, BASELINE_DURATION, BASELINE_PEAK)
            else:
                random_direction = np.random.randn(3)
                random_direction /= np.linalg.norm(random_direction)
                random_peak = np.random.uniform(*RANDOM_PEAK_BOUNDS) 
                random_duration = 30.0 / random_peak * np.random.uniform(*RANDOM_DURATION_BOUNDS)
                force_curve = ForceCurve(random_direction, random_duration, random_peak)

            timer.reset()
            last_update_time = timer.t()
            force_curve.start()
            next_stage()
        elif stage == 2: # trial recording
            t = timer.t()
            dt = timer.dt()
            position = r.get_pose(AXES)
            velocity = r.get_velocity(AXES)
            external_force = r.get_force(AXES)

            force, is_done = force_curve.get_force()

            phrase_trial_data.append(
                t,
                dt,
                position,
                velocity,
                external_force,
                force,
            )

            virtual_dynamics.apply_force(force + external_force, dt)
            r.set_velocity(virtual_dynamics.v, AXES, acceleration=10.0)

            if not is_done and np.linalg.norm(position - HOME_POSE[:3]) > RADIUS:
                force_curve.stop()
                is_done = True

            if is_done:
                r.set_velocity(Robot.zeroed_translation_rotation(), AXES, acceleration=10)
                virtual_dynamics.v *= 0.0
                next_stage()
        elif stage == 3: # get user feedback, restart, or exit
            cmd = input()
            if cmd == 'exit':
                break
            elif cmd == 'restart':
                pass
            elif cmd == '':
                continue
            else:
                phrase_text, adverb, first_cartesian_direction, second_cartesian_direction = cmd.split(',')
                phrase_trial_data.phrase = phrase_text
                phrase_trial_data.adverb = adverb
                phrase_trial_data.first_cartesian_direction = first_cartesian_direction
                phrase_trial_data.second_cartesian_direction = second_cartesian_direction
                phrase_trial_data.force_curve = force_curve
                phrase_trial_data.save(current_trial_number, 'force2lang_trial_data')
                current_trial_number += 1
            next_stage()
        elif stage == 4: # return to home pose
            position = r.get_pose(AXES)
            position_delta = Robot.get_axes(r.INIT_POSE, AXES) - position
            position_delta_magnitude = np.linalg.norm(position_delta)
            position_delta_normalized = position_delta / position_delta_magnitude if position_delta_magnitude != 0 else 0
            return_velocity = 0.2
            r.set_velocity(return_velocity*position_delta_normalized, AXES, acceleration=2)
            if position_delta_magnitude < 0.001:
                r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
                next_stage()

        r.control.waitPeriod(period_start)

print(current_trial_number)

In [ ]:
# ANGLE = 5.0 * np.pi / 4.0
# TRANSFORMATION = np.array([
#     [np.cos(ANGLE), -np.sin(ANGLE), 0.0],
#     [np.sin(ANGLE),  np.sin(ANGLE), 0.0],
#     [0.0, 0.0, 1.0]
# ])

# p = PhraseTrialData.load('./force2lang_trial_data/1__7__forward_and_left.pkl', TRANSFORMATION)

# import matplotlib.pyplot as plt

# # Create the plot
# plt.figure(figsize=(8, 5))
# plt.plot(p.time, p.internal_force, label="Smth over Time")

# # Add labels and title
# plt.xlabel("Time (s)")
# plt.ylabel("Smth")
# plt.title("Smth vs. Time")
# plt.legend()

# # Display the grid and the plot
# plt.grid(True)
# plt.show()